### Packages(Dependency) installation

In [1]:
!pip install diffusers transformers accelerate scipy safetensors --upgrade

  Using cached diffusers-0.34.0-py3-none-any.whl.metadata (20 kB)
  Using cached transformers-4.53.0-py3-none-any.whl.metadata (39 kB)
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using c

### Packages Imports

In [2]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os
from datetime import datetime

### Checking GPU Availability

In [3]:
import torch
# For checking if GPU is available
if not torch.cuda.is_available(): #If Torch discovers no GPU
    raise RuntimeError("GPU Unavailable, Ensure PyTorch was installed with CUDA support.")
    #Raises a Runtime Error stating that the local device Has no CUDA enabled GPU

print("CUDA Available", torch.cuda.is_available()) #If CUDA is available prints "CUDA Available"
print("Using device:", torch.cuda.get_device_name()) #If CUDA is available prints the CUDA device being used
print("PyTorch version:", torch.__version__) #Prints the version of Torch being used
print("GPU Memory Summary:", torch.cuda.memory_summary()) #Prints the summary of the GPU memory

CUDA Available True
Using device: NVIDIA GeForce RTX 4070 Laptop GPU
PyTorch version: 2.7.1+cu126
GPU Memory Summary: |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         

### Choosing the Hardware (CPU/CUDA-GPU)

In [4]:
device = "cuda"

### Using Hugging Face's runwayml/stable-diffusion-v1-5
#### This will download the model the first time (about 4GB)

In [5]:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
pipe = pipe.to(device)

#Optional: Turn off safety checker
pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:  49%|####9     | 1.18G/2.40G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:  23%|##2       | 1.00G/4.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/66/6f/666f465fa70158515404e8de2c6bc6fe2f90c46f9296293aa14daededeb32c52/19da7aaa4b880e59d56843f1fcb4dd9b599c28a1d9d9af7c1143057c8ffae9f1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.safetensors%3B+filename%3D%22diffusion_pytorch_model.safetensors%22%3B&Expires=1751445740&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTQ0NTc0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82Ni82Zi82NjZmNDY1ZmE3MDE1ODUxNTQwNGU4ZGUyYzZiYzZmZTJmOTBjNDZmOTI5NjI5M2FhMTRkYWVkZWRlYjMyYzUyLzE5ZGE3YWFhNGI4ODBlNTlkNTY4NDNmMWZjYjRkZDliNTk5YzI4YTFkOWQ5YWY3YzExNDMwNTdjOGZmYWU5ZjE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=qL-5w58vFsWoYLqRl7lg6mfBnGVcmQtFA9TWgUwnKpWnkvk%7ENeIO6vbsLQjXMeLb0LkyLhdwQ28RHXavx1ugbmpFFf5AYxrIIhvUFVSFC42Tdw9ypIOTwrn7C2k-rpDRRbLl%7EPPvtjFYFujwGLSpSCT5gnZp4nInbwrB8tzqiVxQkbBEA3dyRUO7N1%7EF0eh37naGrEjZRqu7S703ahODcF3DutIF9ep3K

diffusion_pytorch_model.safetensors:  37%|###6      | 2.02G/5.46G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/66/6f/666f465fa70158515404e8de2c6bc6fe2f90c46f9296293aa14daededeb32c52/19da7aaa4b880e59d56843f1fcb4dd9b599c28a1d9d9af7c1143057c8ffae9f1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.safetensors%3B+filename%3D%22diffusion_pytorch_model.safetensors%22%3B&Expires=1751445740&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTQ0NTc0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82Ni82Zi82NjZmNDY1ZmE3MDE1ODUxNTQwNGU4ZGUyYzZiYzZmZTJmOTBjNDZmOTI5NjI5M2FhMTRkYWVkZWRlYjMyYzUyLzE5ZGE3YWFhNGI4ODBlNTlkNTY4NDNmMWZjYjRkZDliNTk5YzI4YTFkOWQ5YWY3YzExNDMwNTdjOGZmYWU5ZjE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=qL-5w58vFsWoYLqRl7lg6mfBnGVcmQtFA9TWgUwnKpWnkvk%7ENeIO6vbsLQjXMeLb0LkyLhdwQ28RHXavx1ugbmpFFf5AYxrIIhvUFVSFC42Tdw9ypIOTwrn7C2k-rpDRRbLl%7EPPvtjFYFujwGLSpSCT5gnZp4nInbwrB8tzqiVxQkbBEA3dyRUO7N1%7EF0eh37naGrEjZRqu7S703ahODcF3DutIF9ep3K

diffusion_pytorch_model.safetensors:  48%|####8     | 3.19G/6.63G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### Define the prompt and generate image

In [19]:
prompt = "A robotic Cat on a building"

num_inference_steps = 30

generated_image = pipe(prompt, num_inference_steps=num_inference_steps).images[0]
generated_image.show()

  0%|          | 0/30 [00:00<?, ?it/s]

Detected locale "C" with character encoding "ANSI_X3.4-1968", which is not UTF-8.
Qt depends on a UTF-8 locale, and has switched to "C.UTF-8" instead.
If this causes problems, reconfigure your locale. See the locale(1) manual
for more information.
